# Modelo 3: LSTM(Bilstm) + CRF + embedding

Realizado por:
- Jose Luis Hincapie Bucheli (2125340)
- Sebastián Idrobo Avirama (2122637)
- Paul Rodrigo Rojas Guerrero (2127891)

---

# Traer información desde Google Drive

Si te encuentras usando Google Colab, probablemente necesites descomentar los siguientes bloques de código y modificar las rutas para que funcione con el tuyo:

In [3]:
# Borrar esto
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sys
import os

sys.path.append('/content/drive/My Drive/Ingeniería de sistemas/Semestre 7/PLN/Taller 3 Personal/packages') # Poner Rutas de los paquetes aquí
sys.path.append('/content/drive/My Drive/Ingeniería de sistemas/Semestre 7/PLN/Taller 3 Personal/embeddings') # Poner Rutas de los embeddings aquí

path = '/content/drive/My Drive/Ingeniería de sistemas/Semestre 7/PLN/Taller 3 Personal/packages'

# Se verifica que todo se encuentre funcionando correctamente
files = os.listdir(path)
print(files)

['wrapper.py', 'tf2crf.py', 'utils.py', 'crfta.py', '__pycache__']


---

# Instalación e importación de paquetes

In [5]:
try:
    import seqeval
except ModuleNotFoundError as err:
    !pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 926.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=7c0fccdb2a32119333e42c1d0a3cfe3f40da40e052a07f0d50933d39e18638f9
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [6]:
!pip install tensorflow-addons==.0.23.0
!pip install tensorflow==2.15.0
!pip install keras==2.15.0
!pip install nltk==3.8.1
!pip install seqeval
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 3.6 MB/s eta 0:00:00


In [7]:
import sys
import os

# Add the directory to the system path
sys.path.append('/packages')

In [8]:
import tensorflow as tf
#matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [9]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
from itertools import islice

#from tabulate import tabulate
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report as eskclarep
#from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score
#from seqeval.metrics import classification_report as seqclarep
from sklearn.preprocessing import LabelBinarizer
from itertools import chain

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Concatenate, Lambda, Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, InputLayer, Activation, Flatten
from tensorflow.keras.optimizers import Adam, schedules
from crfta import CRF as crf4
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard

from IPython.core.display import display, HTML

import datetime, os
import random
import seqeval

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


# Importación de dataset conll2002

In [10]:
import nltk
nltk.download('conll2002')
nltk.corpus.conll2002.fileids()

[nltk_data] Downloading package conll2002 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2002.zip.


['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [11]:
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))
eval_sents = list(nltk.corpus.conll2002.iob_sents('esp.testa'))
print(len(train_sents),len(max(train_sents,key=len)))
print(len(test_sents),len(max(test_sents,key=len)))
print(len(eval_sents),len(max(eval_sents,key=len)))

8323 1238
1517 202
1915 141


In [12]:
print(train_sents[0])

[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]


# Parte 1: Preprocesamiento de los datos

En esta parte, los datos que serán utilizados -conll2002- deben de ser procesados de tal manera que los tokens y labels sean separados y accedidos por medio de listas de índices y convertir las sentencias de representaciones textuales a secuencias de índices.

In [13]:
def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [14]:
X_train = [sent2tokens(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2tokens(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

X_eval = [sent2tokens(s) for s in eval_sents]
y_eval = [sent2labels(s) for s in eval_sents]

In [15]:
print(X_train[2])
print(y_train[2])

['El', 'Abogado', 'General', 'del', 'Estado', ',', 'Daryl', 'Williams', ',', 'subrayó', 'hoy', 'la', 'necesidad', 'de', 'tomar', 'medidas', 'para', 'proteger', 'al', 'sistema', 'judicial', 'australiano', 'frente', 'a', 'una', 'página', 'de', 'internet', 'que', 'imposibilita', 'el', 'cumplimiento', 'de', 'los', 'principios', 'básicos', 'de', 'la', 'Ley', '.']
['O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O']


In [16]:
import numpy as np

words, tagsss = set([]), set([])

for s in (X_train + X_eval + X_test):
    for w in s:
        words.add(w.lower())

for ts in (y_train + y_eval + y_test):
    for t in ts:
        tagsss.add(t)

word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs

tag2index = {t: i + 2 for i, t in enumerate(list(tagsss))}
tag2index['-PAD-'] = 0  # The special value used to padding
tag2index['-OOV-'] = 1  # The special value used to padding

print (len(word2index))
print (len(tag2index))
print(tag2index)


print(tagsss)

28384
11
{'O': 2, 'I-MISC': 3, 'I-PER': 4, 'B-ORG': 5, 'I-ORG': 6, 'B-PER': 7, 'I-LOC': 8, 'B-MISC': 9, 'B-LOC': 10, '-PAD-': 0, '-OOV-': 1}
{'O', 'I-MISC', 'I-PER', 'B-ORG', 'I-ORG', 'B-PER', 'I-LOC', 'B-MISC', 'B-LOC'}


In [17]:
train_sentences_X, eval_sentences_X, test_sentences_X, train_tags_y, eval_tags_y, test_tags_y = [], [], [], [], [], []

for s in X_train:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])

    train_sentences_X.append(s_int)

for s in X_eval:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])

    eval_sentences_X.append(s_int)

for s in X_test:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])

    test_sentences_X.append(s_int)

for s in y_train:
    s_int = []
    for w in s:
        try:
            s_int.append(tag2index[w])
        except KeyError:
            s_int.append(tag2index['-OOV-'])

    train_tags_y.append(s_int)

for s in y_eval:
    s_int = []
    for w in s:
        try:
            s_int.append(tag2index[w])
        except KeyError:
            s_int.append(tag2index['-OOV-'])

    eval_tags_y.append(s_int)

for s in y_test:
    s_int = []
    for w in s:
        try:
            s_int.append(tag2index[w])
        except KeyError:
            s_int.append(tag2index['-OOV-'])

    test_tags_y.append(s_int)


In [18]:
print("Longitudes de las Matrices:")
print(len(train_sentences_X))
print(len(eval_sentences_X))
print(len( test_sentences_X))
print(len(train_tags_y))
print(len(eval_tags_y))
print(len(test_tags_y))

print("\nMuestra de Datos presentes en las Matrices con las transformaciones:\n")


print(train_sentences_X[0])
print(eval_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(eval_tags_y[0])
print(test_tags_y[0])


Longitudes de las Matrices:
8323
1915
1517
8323
1915
1517

Muestra de Datos presentes en las Matrices con las transformaciones:

[15303, 23871, 7994, 21306, 12441, 21329, 1012, 23871, 22192, 21306, 16006]
[27496, 1750, 23871, 1443, 21306, 12441, 4571, 1012, 23871, 9214, 21306, 16006]
[17142, 19304, 12441, 4571, 1012, 23871, 9214, 21306, 16006]
[10, 2, 10, 2, 2, 2, 2, 2, 5, 2, 2]
[10, 8, 2, 10, 2, 2, 2, 2, 2, 5, 2, 2]
[10, 8, 2, 2, 2, 2, 5, 2, 2]


Una vez con estas variables, se debe de normalizar las matrices a partir de MAX_LENGTH, requiriendo de un array estático para servir como input de la RNN.

In [19]:

MAX_LENGTH=202
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
eval_sentences_X = pad_sequences(eval_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
eval_tags_y = pad_sequences(eval_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')

print(train_sentences_X[0])
print(train_sentences_X.shape)
print(eval_sentences_X[0])
print(eval_sentences_X.shape)
print(test_sentences_X[0])
print(test_sentences_X.shape)
print(train_tags_y[0])
print(train_tags_y.shape)
print(eval_tags_y[0])
print(eval_tags_y.shape)
print(test_tags_y[0])
print(test_tags_y.shape)



[15303 23871  7994 21306 12441 21329  1012 23871 22192 21306 16006     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

Y una vez normalizadas las matrices, se procede a representar las listas de indices de etiquetas a una representación one-hot para el procesamiento con el RNN

In [20]:
def to_categoricals(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [21]:
def encode(data):
    print('Shape of data (BEFORE encode): %s' % str(data.shape))
    encoded = to_categorical(data)
    print('Shape of data (AFTER  encode): %s\n' % str(encoded.shape))
    return encoded

In [22]:
cat_train_tags_y = to_categoricals(train_tags_y, len(tag2index))
cat_eval_tags_y  = to_categoricals(eval_tags_y, len(tag2index))
cat_test_tags_y  = to_categoricals(test_tags_y, len(tag2index))

print(cat_train_tags_y[1])
print(len(cat_train_tags_y))
print(cat_train_tags_y.shape)
print(len(cat_test_tags_y))

[[0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
8323
(8323, 202, 11)
1517


# Parte 2: Entrenamiento del modelo de red

Se define la red neuronal para el trabajo de NER usando el modelo LSTM(Bilstm) + CRF + embedding, importando los módulos necesarios para el mismo y también parametrizando la RNN a entrenar.
- Se trae el embedding con tamaño de 300.
- Se genera el modelo LSTM(Bilstm) + CRF
- Se genera una capa de masking para que ignore los valores de 0

Una vez hecho esto, se procede a entrenar la red neuronal y a testear los resultados de la misma

In [23]:
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df
EMBED_DIM = 300

embedding_file = "/content/drive/MyDrive/Ingeniería de sistemas/Semestre 7/PLN/Taller 3 Personal/embeddings/word2vec_jose_sebastian_paul300.txt"

embedding_matrix = bme(embedding_file, len(word2index), EMBED_DIM, word2index)


Cargando archivo...


0it [00:00, ?it/s]

Encontrado 220714 Word Vectors.


  0%|          | 0/28384 [00:00<?, ?it/s]

Convertidos: 17841 Tokens | Perdidos: 10543 Tokens


In [30]:
# TO-DO: No se ha hecho nada diferente por el momento

from tf2crf import CRF as crf6
from wrapper import ModelWithCRFLoss, ModelWithCRFLossDSCLoss
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df
from tensorflow.keras.layers import Concatenate, Lambda, Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, InputLayer, Activation, Flatten, Masking
from tensorflow.keras.optimizers import Adam, schedules
input = Input(shape=(MAX_LENGTH,))
word_embedding_size = 300
EMBED_DIM=300
# Embedding Layer
model = Embedding(input_dim=len(word2index),
                weights=[embedding_matrix],
                output_dim=word_embedding_size,
                input_length=MAX_LENGTH,
                mask_zero=False)(input)

# BI-LSTM Layer
model = Bidirectional(LSTM(units=50,
                     return_sequences=True,
                     dropout=0.5,
                     recurrent_dropout=0.5))(model)

# LSTM(Bilstm) Layer
model = LSTM(units=50,
                     return_sequences=True,
                     dropout=0.5,
                     recurrent_dropout=0.5)(model)

model  = Dropout(0.5, name='dropout_lstm')(model)
model  = Dense(units=EMBED_DIM * 2, activation='relu')(model)
model  = Dense(units=len(tag2index), activation='relu')(model)

model  = Masking(mask_value=0.,input_shape=(MAX_LENGTH, len(tag2index)))(model)

crf = crf6(units=len(tag2index), name="ner_crf")
predictions = crf(model)

base_model = Model(inputs=input, outputs=predictions)
model = ModelWithCRFLoss(base_model, sparse_target=True)

model.compile(optimizer='adam')
#model.summary()

In [31]:
history= model.fit(train_sentences_X, cat_train_tags_y,
                       validation_data=(eval_sentences_X, cat_eval_tags_y),
                       batch_size=128,
                       epochs=50,
                       verbose=2)

Epoch 1/50
66/66 - 53s - loss: 172.2674 - accuracy: 0.7290 - val_loss_val: 57.9356 - val_val_accuracy: 0.9099 - 53s/epoch - 799ms/step
Epoch 2/50
66/66 - 40s - loss: 40.8134 - accuracy: 0.9506 - val_loss_val: 23.5037 - val_val_accuracy: 0.9783 - 40s/epoch - 608ms/step
Epoch 3/50
66/66 - 40s - loss: 20.2340 - accuracy: 0.9792 - val_loss_val: 17.5094 - val_val_accuracy: 0.9798 - 40s/epoch - 609ms/step
Epoch 4/50
66/66 - 40s - loss: 15.6926 - accuracy: 0.9805 - val_loss_val: 14.9787 - val_val_accuracy: 0.9808 - 40s/epoch - 611ms/step
Epoch 5/50
66/66 - 40s - loss: 13.7220 - accuracy: 0.9812 - val_loss_val: 13.6973 - val_val_accuracy: 0.9812 - 40s/epoch - 606ms/step
Epoch 6/50
66/66 - 40s - loss: 12.2188 - accuracy: 0.9816 - val_loss_val: 13.1569 - val_val_accuracy: 0.9813 - 40s/epoch - 607ms/step
Epoch 7/50
66/66 - 40s - loss: 11.0957 - accuracy: 0.9821 - val_loss_val: 12.6971 - val_val_accuracy: 0.9818 - 40s/epoch - 610ms/step
Epoch 8/50
66/66 - 41s - loss: 10.5756 - accuracy: 0.9827 - v

In [32]:
print(tag2index)
print(test_sentences_X)
y_pred= model.predict(test_sentences_X)
print(y_pred.shape)
print(y_pred)

{'O': 2, 'I-MISC': 3, 'I-PER': 4, 'B-ORG': 5, 'I-ORG': 6, 'B-PER': 7, 'I-LOC': 8, 'B-MISC': 9, 'B-LOC': 10, '-PAD-': 0, '-OOV-': 1}
[[17142 19304 12441 ...     0     0     0]
 [19435     0     0 ...     0     0     0]
 [ 5672  2757 24200 ...     0     0     0]
 ...
 [17142   104 18572 ...     0     0     0]
 [ 5831 12711  9195 ...     0     0     0]
 [17142  1023 14711 ...     0     0     0]]
48/48 [==============================] - 5s 80ms/step
(1517, 202)
[[10  8  2 ...  0  0  0]
 [ 2  0  0 ...  0  0  0]
 [ 2  2  2 ...  0  0  0]
 ...
 [ 2 10  2 ...  0  0  0]
 [ 2  2  2 ...  0  0  0]
 [ 2  2  2 ...  0  0  0]]


In [33]:
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df
index2tag = {i: t for t, i in tag2index.items()}
print(index2tag)
y1_pred = logits_to_tokens(y_pred, index2tag)
print(y1_pred[10])

{2: 'O', 3: 'I-MISC', 4: 'I-PER', 5: 'B-ORG', 6: 'I-ORG', 7: 'B-PER', 8: 'I-LOC', 9: 'B-MISC', 10: 'B-LOC', 0: '-PAD-', 1: '-OOV-'}
['O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-'

In [34]:
#print(Y_test[4])
print(test_tags_y.shape)

(1517, 202)


In [35]:
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df
index2tag = {i: t for t, i in tag2index.items()}
print(index2tag)
y1_true = logits_to_tokens(test_tags_y, index2tag)
print(y1_true[10])

{2: 'O', 3: 'I-MISC', 4: 'I-PER', 5: 'B-ORG', 6: 'I-ORG', 7: 'B-PER', 8: 'I-LOC', 9: 'B-MISC', 10: 'B-LOC', 0: '-PAD-', 1: '-OOV-'}
['O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-'

## Calidad obtenida del modelo

Se evalua la calidad de los resultados del modelo usando las siguientes métricas: *precision, recall, accuracy* y *F1-score*.

In [36]:
from seqeval.metrics import classification_report as seqclarep
from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score
print("precision: {:.1%}".format(precision_score(y1_true, y1_pred)))
print("   recall: {:.1%}".format(recall_score(y1_true,    y1_pred)))
print(" accuracy: {:.1%}".format(accuracy_score(y1_true,  y1_pred)))
print(" F1-score: {:.1%}".format(f1_score(y1_true,        y1_pred)))

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: -PAD- seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


precision: 67.4%
   recall: 64.9%
 accuracy: 99.1%
 F1-score: 66.1%


Se genera un reporte de la clasificación basada en las etiquetas predichas y las reales, esto brindado en un DataFrame.

In [37]:
import pandas as pd
li1 = sum(y1_true, [])
li2 = sum(y1_pred, [])

results = pd.DataFrame(columns=['Expected', 'Predicted'])

results['Expected'] = li1
results['Predicted'] = li2

In [38]:
from sklearn.metrics import classification_report as eskclarep
report = eskclarep(results['Expected'], results['Predicted'])

print(report_to_df(report))

  Class Name precision recall f1-score support
0      -PAD-      1.00   1.00     1.00  254901
1      B-LOC      0.76   0.70     0.73    1084
2     B-MISC      0.32   0.35     0.33     339
3      B-ORG      0.73   0.78     0.76    1400
4      B-PER      0.90   0.66     0.76     735
5      I-LOC      0.63   0.61     0.62     325
6     I-MISC      0.49   0.43     0.46     557
7      I-ORG      0.68   0.68     0.68    1104
8      I-PER      0.94   0.76     0.84     634
9          O      0.97   0.98     0.98   45355


Por último, se realizan evaluaciones del modelo a partir de algunas oraciones.

In [39]:
test_sample1 = [
    "James Rodriguez es el jugador colombiano más importante con Radamel Falcao.".split(),
    " Jugadores de la selección Colombia que juegan en el Reino Unido".split()
]

print(test_sample1)

test_sample2 = [
    "La federación Nacional de cafeteros de Colombia es dirigida por Horacio Sánchez".split(),
    " y se ubica en las ciudades de  Cali y Medellín con el instituto colombiano del café ".split()
]

print(test_sample2)

[['James', 'Rodriguez', 'es', 'el', 'jugador', 'colombiano', 'más', 'importante', 'con', 'Radamel', 'Falcao.'], ['Jugadores', 'de', 'la', 'selección', 'Colombia', 'que', 'juegan', 'en', 'el', 'Reino', 'Unido']]
[['La', 'federación', 'Nacional', 'de', 'cafeteros', 'de', 'Colombia', 'es', 'dirigida', 'por', 'Horacio', 'Sánchez'], ['y', 'se', 'ubica', 'en', 'las', 'ciudades', 'de', 'Cali', 'y', 'Medellín', 'con', 'el', 'instituto', 'colombiano', 'del', 'café']]


In [40]:
test_samples_X = []
for s in test_sample2:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)

test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
print(test_samples_X)
print(test_samples_X.shape)

[[17142  4694  3923 16842 19032 16842 21141 15998  9827 16156     1  3582
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0 

In [41]:
predictions = model.predict(test_samples_X)
print(predictions, predictions.shape)

1/1 [==============================] - 0s 83ms/step
[[ 2  5  6  6  6  6  6  2  2  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]
 [ 2  2  2  2  2  2  2 10  2  2  2  2  2  2  2  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  

In [42]:
#print(len(predictions))
log_tokens = logits_to_tokens(predictions, {i: t for t, i in tag2index.items()})
print(log_tokens)

[['O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-

In [43]:
!pip install tabulate
from tabulate import tabulate

heads1 = test_sample2[0]
body1 = [log_tokens[0][:len(test_sample2[0])]]

heads2 = test_sample2[1]
body2 = [log_tokens[1][:len(test_sample2[1])]]

print(tabulate(body1, headers=heads1))

print ("\n")

print(tabulate(body2, headers=heads2))

## postagging Freeling 4.1

## El      hombre   bajo     corre    bajo  el      puente   con  bajo  índice   de  adrenalina  .
## DA0MS0  NCMS000  AQ0MS00  VMIP3S0  SP    DA0MS0  NCMS000  SP   SP    NCMS000  SP  NCFS000     Fp


## pos tagger Stanford NLP

## El      hombre   bajo     corre    bajo  el      puente   con    bajo   índice  de    adrenalina  .
## da0000  nc0s000  aq0000   vmip000  sp000 da0000  nc0s000  sp000  aq0000 nc0s000 sp000 nc0s000     fp

La    federación    Nacional    de     cafeteros    de     Colombia    es    dirigida    por    Horacio    Sánchez
----  ------------  ----------  -----  -----------  -----  ----------  ----  ----------  -----  ---------  ---------
O     B-ORG         I-ORG       I-ORG  I-ORG        I-ORG  I-ORG       O     O           O      -PAD-      -PAD-


y    se    ubica    en    las    ciudades    de    Cali    y    Medellín    con    el    instituto    colombiano    del    café
---  ----  -------  ----  -----  ----------  ----  ------  ---  ----------  -----  ----  -----------  ------------  -----  ------
O    O     O        O     O      O           O     B-LOC   O    O           O      O     O            O             O      -PAD-
